In [1]:
!pip install emoji
!pip install -q pandarallel
!sudo apt-get -o Dpkg::Options::="--force-confmiss" install --reinstall netbase
!pip install git+https://github.com/crazyfrogspb/RedditScore.git
!pip install extraction

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=2564595cb88616b23a3605e315181e9c1de76d46f3b2a3621de8136714997650
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  netbase
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 12.7 kB of archives.
After this operation, 45.1 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 netbase all 5.4 [12.7 kB]
Fetched 12.7 kB in 0s (47.6 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling

In [2]:
import pandas as pd
import sys
import json
import os
import extraction
import requests
from google.colab import drive
from redditscore.tokenizer import CrazyTokenizer
import nltk
import re
import emoji
import requests
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import warnings
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('punkt') # if necessary...
pd.set_option('display.max_colwidth', 255)
warnings.filterwarnings("ignore")
nltk.download('stopwords')
drive.mount('/content/drive')

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/COVID19 Fake News Detection in English/CoAID/'

In [ ]:
real1 = pd.read_json(f'{path}09-01-2020/09-01-2020real_tweet.json')
real1['label'] = 'real'
real2 = pd.read_json(f'{path}07-01-2020/07-01-2020real_tweet.json')
real2['label'] = 'real'
real3 = pd.read_json(f'{path}05-01-2020/05-01-2020real_tweet.json')
real3['label'] = 'real'

In [ ]:
len(real1), len(real2), len(real3)

(104, 408, 9129)

In [ ]:
fake1 = pd.read_json(f'{path}09-01-2020/09-01-2020fake_tweet.json')
fake1['label'] = 'fake'
fake2 = pd.read_json(f'{path}07-01-2020/07-01-2020fake_tweet.json')
fake2['label'] = 'fake'
fake3 = pd.read_json(f'{path}05-01-2020/05-01-2020fake_tweet.json')
fake3['label'] = 'fake'

In [ ]:
len(fake1), len(fake2), len(fake3)

(23, 1116, 5349)

In [ ]:
data = pd.concat([
    real1[['text', 'label']],
    real2[['text', 'label']],
    real3[['text', 'label']],
    fake1[['text', 'label']],
    fake2[['text', 'label']],
    fake3[['text', 'label']]],axis = 0).reset_index(drop=True)
data = data.sample(frac=1).reset_index(drop=True)

In [ ]:
data = data.rename(columns={'text': 'tweet'})

In [ ]:
def without_link_tweet(text):
  raw_text = re.sub(r'(https?://[^\s]+)', '', text, flags=re.MULTILINE)
  return raw_text
data['raw_tweet'] = data.parallel_apply(lambda x: without_link_tweet(x['tweet']), axis=1)

   0.00%                                          |        0 /     8065 |      
   0.00%                                          |        0 /     8064 |      M   0.99%                                          |       80 /     8065 |      
   0.00%                                          |        0 /     8064 |      M   5.11% ::                                       |      412 /     8065 |      
   0.99%                                          |       80 /     8064 |      M   5.11% ::                                       |      412 /     8065 |      
  83.28% :::::::::::::::::::::::::::::::::        |     6716 /     8064 |      M 100.00% :::::::::::::::::::::::::::::::

In [ ]:
data = data.drop_duplicates(subset=['raw_tweet'],keep=False)
data = data.reset_index(drop=True)

In [ ]:
data.label.value_counts(), len(data)

(real    4727
 fake    3412
 Name: label, dtype: int64, 8139)

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in stopwords.words('english')])
data['raw_tweet'] = data['raw_tweet'].apply(lambda s : text_process(s))
data = data.drop_duplicates(subset=['raw_tweet'],keep=False)
data = data.reset_index(drop=True)

In [ ]:
data.label.value_counts(), len(data)

(real    4129
 fake    3162
 Name: label, dtype: int64, 7291)

In [ ]:
data

,tweet,label,raw_tweet
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt
2,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)",real,Second Wave COVID19 First Never Ended Many Americans wonder coming months pandemic look like states case rates currently high well states the… Feed generated FetchRSS
3,Emergency Report! The Coronavirus Hoax Ends Human Civilization https://t.co/iTlBrPcugW #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #...,fake,Emergency Report Coronavirus Hoax Ends Human Civilization SayNoToBillGates NoNewNormal COVID19 Coronavirus Plandemic Scamdemic Fakedemic JailBillGates FuckBillGates BigTechIsEvil 5G Crypto ID2020 Agenda2030 Depopulation WakeUp
4,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years.",real,Good evidence slow Covid19 mutation rate Bodes well vaccines development people get vaccinated 2021 new coronavirus may well enjoy protection lasts years
...,...,...,...
7286,"The funded projects include 11 community-led initiatives to address Wisconsin community health needs and ten #UWMadison research grants to lessen the impact of COVID-19 through improved testing, treatments and vaccine development. @WisPartnership\n🔬🧪🩺...",real,funded projects include 11 communityled initiatives address Wisconsin community health needs ten UWMadison research grants lessen impact COVID19 improved testing treatments vaccine development WisPartnership 🔬🧪🩺
7287,Get it togedffffffffcvhjj\n\nThe CDC Confesses to Lying About COVID-19 Death Numbers https://t.co/vPUi1n3JIw,fake,Get togedffffffffcvhjj CDC Confesses Lying COVID19 Death Numbers
7288,"NEW: “If you don’t test, you don’t have any cases,” Trump said at a June 15 roundtable discussion at the White House. “If we stopped testing right now, we’d have very few cases, if any.”\n\n📝: @shefalil, @KHNews/@PolitiFact #HealthCheck https://t.co/Z...",fake,NEW “If don’t test don’t cases” Trump said June 15 roundtable discussion White House “If stopped testing right we’d cases any” 📝 shefalil KHNewsPolitiFact HealthCheck
7289,COVID-19: The Immune System Can Fight Back https://t.co/UTsavTxqqr via @DubaiChronicle,real,COVID19 Immune System Fight Back via DubaiChronicle


In [ ]:
stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]
print(cosine_sim('politically correct woman (almost) uses pandemic as excuse not to reuse plastic bag  #coronavirus #nashville', 'politically correct woman almost uses pandemic as excuse not to reuse plastic bag | the spoof'))
#print(cosine_sim('a little bird', 'None'))
# print(cosine_sim('a little bird', 'a big dog barks'))

0.7548969126692828


In [3]:
path = '/content/drive/MyDrive/COVID19 Fake News Detection in English/input/'
'''
def _store_data(load_data,link):
  store_path = f"{path}{link}.json"
  if os.path.exists(store_path) == False:
      store_data = []
      store_data.append(load_data)
      with open(store_path, 'w') as fp:
          json.dump(store_data, fp)
  else:
      with open(store_path, 'r') as fp:
          data = json.load(fp)
      data.append(load_data)
      with open(store_path, 'w') as fp:
          json.dump(data, fp)
  return

def text_replace(df,link):
  len_df = len(df)
  for index in range(len_df):
    text = df['tweet'][index]
    urls = re.findall('(https?://[^\s]+)',  df['tweet'][index])
    data = dict()
    data['index'] = index
    for url in urls:
      try:
        url = re.findall('.*[^.]', url)[0]
        html = requests.get(url).text
        extracted = extraction.Extractor().extract(html, source_url=url)
        data[url] = extracted.title
      except:
        continue
    if index % 500 == 0:
      print(index)
    _store_data(data,link)
text_replace(data,"additional_link_store")
'''

'\ndef _store_data(load_data,link):\n  store_path = f"{path}{link}.json"\n  if os.path.exists(store_path) == False:\n      store_data = []\n      store_data.append(load_data)\n      with open(store_path, \'w\') as fp:\n          json.dump(store_data, fp)\n  else:\n      with open(store_path, \'r\') as fp:\n          data = json.load(fp)\n      data.append(load_data)\n      with open(store_path, \'w\') as fp:\n          json.dump(data, fp)\n  return\n\ndef text_replace(df,link):\n  len_df = len(df)\n  for index in range(len_df):\n    text = df[\'tweet\'][index]\n    urls = re.findall(\'(https?://[^\\s]+)\',  df[\'tweet\'][index])\n    data = dict()\n    data[\'index\'] = index\n    for url in urls:\n      try:\n        url = re.findall(\'.*[^.]\', url)[0]\n        html = requests.get(url).text\n        extracted = extraction.Extractor().extract(html, source_url=url)\n        data[url] = extracted.title\n      except:\n        continue\n    if index % 500 == 0:\n      print(index)\n    _

In [ ]:
def politifact_count(link):
  path = f"/content/drive/My Drive/COVID19 Fake News Detection in English/input/{link}.json"
  with open(path, 'r') as fp:
    data = json.load(fp)
  politifact = 0
  poli_list = []
  for obj in data:
    keys = list(obj.keys())
    keys.remove('index')
    if len(keys) > 0:
      for url in keys:
        if obj[url] != None:
          text = obj[url].lower()
          text = text.split(' ')
          if 'politifact' in text:
            poli_list.append(obj['index'])
            politifact += 1
  return poli_list,len(data), politifact

data_poli_list,data_len, data_poli = politifact_count("additional_link_store")
print(f"politifact website exist in additional data is = {data_poli/data_len * 100}num is {data_poli}/{data_len}")

politifact website exist in additional data is = 0.576052667672473num is 42/7291


In [ ]:
data['clean_tweet'] = data['tweet']

In [ ]:
def link_replace(df, link):
  path = f"/content/drive/My Drive/COVID19 Fake News Detection in English/input/{link}.json"
  with open(path, 'r') as fp:
    data = json.load(fp)
  len_df = len(df)
  for index in range(len_df):
    raw_text = re.sub(r'(https?://[^\s]+)', '', df['clean_tweet'][index], flags=re.MULTILINE)
    urls = re.findall('(https?://[^\s]+)',df['clean_tweet'][index])
    if len(urls) > 0:
      for obj in data:
        if index == obj['index']:
          for url in urls:
            try:
              location_text = obj[url]
              if location_text == None:
                df['clean_tweet'][index] = re.sub(url, '', df['clean_tweet'][index])
              elif cosine_sim(raw_text.lower(), location_text.lower()) > 0.7:
                df['clean_tweet'][index] = re.sub(url, '', df['clean_tweet'][index])
              else:
                df['clean_tweet'][index] = re.sub(url, location_text, df['clean_tweet'][index])
            except:
              pass
  return df
data = link_replace(data,"additional_link_store")

In [ ]:
data

,tweet,label,raw_tweet,clean_tweet
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? Does Your Blood Type Increase Your Risk for Coronavirus? by @healthline
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; What It's Like to Be a Nurse Working on the Front Lines of COVID-19
2,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)",real,Second Wave COVID19 First Never Ended Many Americans wonder coming months pandemic look like states case rates currently high well states the… Feed generated FetchRSS,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)"
3,Emergency Report! The Coronavirus Hoax Ends Human Civilization https://t.co/iTlBrPcugW #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #...,fake,Emergency Report Coronavirus Hoax Ends Human Civilization SayNoToBillGates NoNewNormal COVID19 Coronavirus Plandemic Scamdemic Fakedemic JailBillGates FuckBillGates BigTechIsEvil 5G Crypto ID2020 Agenda2030 Depopulation WakeUp,Emergency Report! The Coronavirus Hoax Ends Human Civilization #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #Depopulation #WakeUp
4,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years.",real,Good evidence slow Covid19 mutation rate Bodes well vaccines development people get vaccinated 2021 new coronavirus may well enjoy protection lasts years,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years."
...,...,...,...,...
7286,"The funded projects include 11 community-led initiatives to address Wisconsin community health needs and ten #UWMadison research grants to lessen the impact of COVID-19 through improved testing, treatments and vaccine development. @WisPartnership\n🔬🧪🩺...",real,funded projects include 11 communityled initiatives address Wisconsin community health needs ten UWMadison research grants lessen impact COVID19 improved testing treatments vaccine development WisPartnership 🔬🧪🩺,"The funded projects include 11 community-led initiatives to address Wisconsin community health needs and ten #UWMadison research grants to lessen the impact of COVID-19 through improved testing, treatments and vaccine development. @WisPartnership\n🔬🧪🩺..."
7287,Get it togedffffffffcvhjj\n\nThe CDC Confesses to Lying About COVID-19 Death Numbers https://t.co/vPUi1n3JIw,fake,Get togedffffffffcvhjj CDC Confesses Lying COVID19 Death Numbers,Get it togedffffffffcvhjj\n\nThe CDC Confesses to Lying About COVID-19 Death Numbers
7288,

In [ ]:
final_hash = []
def hash_tag_count(df):
  hash_tag_words = []
  hash_tag = df.clean_tweet.str.findall(r'#.*?(?=\s|$)')
  for val in hash_tag:
    if (len(val) != 0):
      words = [word for word in val]
      hash_tag_words.extend(words)
  return hash_tag_words, Counter(hash_tag_words), len(hash_tag_words)
test_hash,hash_tag_words_test, count_test = hash_tag_count(data)
print(hash_tag_words_test)
print(count_test)
final_hash.extend(test_hash)

Counter({'#coronavirus': 459, '#COVID19': 339, '#Coronavirus': 164, '#COVID-19': 83, '#covid19': 82, '#Covid_19': 73, '#COVIDー19': 59, '#health': 57, '#CDC': 50, '#science': 48, '#CoronavirusOutbreak': 42, '#livenewsindia': 42, '#vaccine': 41, '#Health': 38, '#Covid19': 38, '#COVID2019': 36, '#flu': 34, '#SmartNews': 33, '#COVID': 32, '#Agenparl': 31, '#CoronavirusPandemic': 31, '#Mask"': 28, '#SurgeonGeneral': 28, '#DrFauci': 28, '#Masks"': 28, '#CoronaVirusUpdate': 27, '#CoronaVirus': 25, '#virus': 25, '#NIH': 25, '#pandemic': 24, '#research': 23, '#news': 23, '#wellness': 22, '#Science': 21, '#AgenparlEnglish': 21, '#Educazione': 21, '#SocialNetwork': 21, '#globalnews': 21, '#newsmedia': 21, '#Flu': 20, '#Salute': 20, '#ScienzaETecnologia': 20, '#HealthNews': 20, '#Vaccine': 19, '#healthy': 19, '#News': 18, '#CoronaVirusUpdates': 17, '#China': 17, '#healthyliving': 16, '#SARSCoV2': 16, '#HealthDiscoveries': 16, '#vaccines': 15, '#primarycare': 15, '#COVID__19': 14, '#preventativemed

In [ ]:
emojis = []
for txt in data['clean_tweet']:
  text = emoji.demojize(txt)
  text = re.findall(r'(:[!_\-\w]+:)', text)
  list_emoji = [emoji.emojize(x) for x in text]
  emojis.extend(list_emoji)

In [ ]:
print(emojis)

['🌟', '🌟', '😏', '😱', '\U0001f92c', '\U0001f92e', '👿', '🤞🏼', '💪🏼', '🙌🏼', '\U0001f929', '❓', '🙄', '🐇', '🇺🇸', '🤔', '🤔', '🤔', '🤔', '🤔', '😐', '\U0001f91f', '⁉', '👁', '🇺🇸', '🇺🇸', '🇺🇸', '🌎', '🌎', '🌎', '😂', '😂', '😂', '😂', '🌎', '🗽', '📊', '🌞', '🙌', '🌱', '🎼', '🍀', '🙏', '🌝', '😨', '😨', '😨', '👍', '🤔', '🤣', '🔽', '👉🏼', '⭕', '😱', '😷', '🇨🇳', '👇', '❌', '❌', '❌', 'ℹ️', '🔗', '😒', '🌕', '❔', '🤗', '👼', '\U0001f92f', '😷', '😲', '🇺🇸', '💪', '🌊', '📝', '🤡', '😜', '😂', '😱', '🏡', '🤦🏻\u200d♂️', '👇', '👇', '👇', '😕', '🇺🇸', '👊', '🤔', '⚠️', '🇺🇸', '🇺🇸', '🇺🇸', '📲', '📲', '📲', '📲', '📲', '📲', '📲', '🇻🇦', '👀', '😂', '🇨🇳', '®', '🤣', '🤣', '🤣', '👆', '😎', '😥', '😥', '😥', '😜', '😜', '😜', '⁉', '✨', '👀', '👇🏼', '🤣', '🤣', '🤣', '🤣', '🤣', '🤣', '🤣', '🇺🇸', '\U0001f92a', '🏆', '👹', '🤢', '\U0001f92e', '😳', '⚠️', '⚠️', '👀', '‼️', '⁉️', '\U0001f92c', '🙏', '🙏', '🙏', '🙏', '👉🏻', '🔗', '🤔', '🙄', '❌', '✊🏽', '❤️', '🇦🇺', '🔴', '🔴', '😳', '👑', '😱', '🤣', '🤣', '🤣', '🇺🇸', '😉', '🌎', '🔊', '💀', '💀', '💀', '™', '✝', '☣', '☣', '☣', '☣', '☣', '➡️', '👉', '🖕', '🖕', '🎥', '😷'

In [ ]:
def replace_emoji(text):
    text = text.lower()
    text = re.sub(r"👑🦠",  " coronavirus ", text)
    text = re.sub(r"📍", " update ", text)
    text = re.sub(r"🆕", " update ", text)
    text = re.sub(r"📌", " update ", text) 
    #text = re.sub(r"✅", " verified", text)
    text = re.sub(r"📢", " update ", text)
    text = re.sub(r"📣", " update ", text)
    text = re.sub(r"🔴", " warning ", text)
    text = re.sub(r"👍", " approved ", text)
    text = re.sub(r"🔰", " driving sign ", text)
    text = re.sub(r"🙏", " thank you ", text)
    text = re.sub(r"😷", " wear a mask ", text)
    text = re.sub(r"❗", " excitement ", text)
    text = re.sub(r"🌍", " europe africa ", text)
    text = re.sub(r"🌎", " america ", text)
    text = re.sub(r"🌏", " asia austrila ", text) 
    text = re.sub(r"🌐", " globe ", text) 
    text = re.sub(r"🏫", " school ", text)
    text = re.sub(r"📺", " television ", text)
    text = re.sub(r"👏", " hand washing ", text)
    text = re.sub(r"🤔", " thinking ", text)
    text = re.sub(r"🏠", " stay at home ", text)
    text = re.sub(r"🏡", " stay at home ", text)
    text = re.sub(r"🕌", " mosque ", text)
    text = re.sub(r"📈", " increasing ", text)
    text = re.sub(r"📉", " decreasing ", text) 
    text = re.sub(r"🙌", " hand washing ", text)
    text = re.sub(r"👐", " hand washing ", text)
    text = re.sub(r"👦", " boy ", text)
    text = re.sub(r"❌", " not ", text)
    text = re.sub(r"🙄", " rolling eye ", text)
    text = re.sub(r"📏", " keep distance ", text)
    text = re.sub(r"👥", " social distance ", text)
    text = re.sub(r"💉", " vaccination ", text)
    text = re.sub(r"\U0001f9ea", "vaccination ", text)
    text = re.sub(r"🧪", "vaccination ", text) 
    text = re.sub(r"💊", " pill ", text)
    text = re.sub(r"🗣️", " speak ", text) 
    text = re.sub(r"🚍", " transportation ", text)
    text = re.sub(r"🚌", " transportation ", text)
    text = re.sub(r"💺", " seat ", text) 
    text = re.sub(r"🐍", " snake ", text)
    text = re.sub(r"💊", " mike ", text)
    text = re.sub(r"🏥", " hospital ", text) 
    text = re.sub(r"🚑", " ambulance ", text)  
    text = re.sub(r"🚨", " emergency ", text)
    text = re.sub(r"🏪", " convenience store ", text)
    text = re.sub(r"😞", " disappointed ", text) 
    text = re.sub(r"🎉", " party popper ",text )
    text = re.sub(r"📰", " newspaper ",text )
    text = re.sub(r"🤱", " breast feeding ",text)
    text = re.sub(r"👀", " astonished ",text)
    text = re.sub(r"🧤", " gloves ",text)
    text = re.sub(r"🚶🏿", " social distance ",text)
    text = re.sub(r"🚥", " traffic light ",text)
    text = re.sub(r"↗", " up right ",text)
    text = re.sub(r"⚕", " medicine and health care services ",text)
    
    # text = re.sub(r"🙂", " smile",text)
    # text = re.sub(r"😁", " smile",text)
    # text = re.sub(r"😠", " angry",text)
    # text = re.sub(r"😭", " crying",text)
    # text = re.sub(r"😧", " worry",text) #anguished
    # text = re.sub(r"😢", " crying",text)
    # text = re.sub(r"😣", " helplessness",text)#persevering_face
    # text = re.sub(r"😞", " disappointed",text)
    # text = re.sub(r"😖", " sad",text)
    # text = re.sub(r"😬", " disapproval",text)
    # text = re.sub(r"🤣", " laughing",text)
    # text = re.sub(r"😐", " neutral",text)
    
    text = re.sub(r"↔️", " keep distance ", text)
    text = re.sub(r"🦠", " virus ", text)
    text = re.sub(r"💧", " droplet ", text)
    text = re.sub(r"💦", " droplet ", text)
    text = re.sub(r"🧴", " sanitizer ", text)
    text = re.sub(r"🧼", " hand wash ", text)
    text = re.sub(r"🚫", " not ", text)
    text = re.sub(r"🚰", " hand wash ", text)
    text = re.sub(r"🤝", " not handshake ", text)
    text = re.sub(r"🧑‍🤝‍🧑", " not close contact ", text)
    text = re.sub(r"🧫", " test for covid-19 ", text)
    text = re.sub(r"🧪", " test for covid-19 ", text)
    text = re.sub(r"🌡️", " body temperature measured ", text)
    text = re.sub(r"🔬", " test for covid-19 ", text)
    text = re.sub(r"📝", " contact tracing ", text)
    text = re.sub(r"🧑‍⚕️", " health worker ", text)
    text = re.sub(r"🐦", " hand wash ", text)
    text = re.sub(r"🛡", " shielding required ", text)
    text = re.sub(r"♿️", " accessibility requirements ", text)
    text = re.sub(r"🆘", " emergency help required ", text)    

    with open('/content/drive/My Drive/COVID19 Fake News Detection in English/country_emoji.json') as json_file:
      data = json.load(json_file)
    for val in data:
      name = f" {val['name']} "
      text = re.sub(val['emoji'], name, text)
    text = text.strip(' ')
    return text
def digit_remove(text):
  text = re.sub("\d+", " ", text)
  #text = text.replace('$$', "19")
  return text

In [ ]:
def preprocess(text):
  tok_words = []
  text = text.lower()
  text = text.split(' ')
  for tok_word in text: 
    if "#covid" in tok_word:
      sub_split_pos = tok_word.find('19')
      if sub_split_pos == -1: #coviduk
        if tok_word.strip() == "#covid":
          tok_words.append("covid-19")
        else:
          tok_words.append("covid-19")
          tok_words.append(tok_word[6:])

      elif sub_split_pos+2 == len(tok_word):
        tok_words.append("covid-19")
        #tok_words.append(tok_word.replace("#",""))

      else: #COVID-19Uganda
        fst = tok_word[:sub_split_pos+2]
        tok_words.append("covid-19")
        #tok_words.append(fst.replace("#",""))
        sec = tok_word[sub_split_pos+2:]
        tok_words.append(sec)
    else:
      tok_words.append(tok_word)

  text = ' '.join(tok_words)
  text = text.replace("19","$$")
  text = digit_remove(text)
  text = replace_emoji(text)
  text = text.strip()
  text = text.replace("$$","19")
  text = text.replace("politifact","")
  text = text.replace("forbidden","")
  text = text.replace("page not found","")
  text = text.replace("bad request","")
  text = text.replace("not acceptable","")
  text = text.replace("access denied | banned","")
  text = text.replace("access denied","")  
  text = ' '.join(text.split()) 
  return text


In [ ]:
t = "Forbidden PolitiFact 9/11 out🆘 of 🆘 13 people🇯🇵 🇳🇴 🇱🇦 🆘 #COVID--19Updates 🤣 #COVID19 25th Sep 2569 #CovidUK #COVID19Pandemic #covid19ireland #COVID19NZ #covid__19 #covid--19 #covid 19 #COVID-19Nigeria #COVID-19"
t = preprocess(t) 
t

'/ out emergency help required of emergency help required people japan norway laos emergency help required covid-19 updates 🤣 covid-19 th sep covid-19 uk covid-19 pandemic covid-19 ireland covid-19 nz covid-19 covid-19 covid-19 19 covid-19 nigeria covid-19'

In [ ]:
data['clean_tweet'] = data['clean_tweet'].apply(lambda s : preprocess(s))

In [ ]:
data.head()

,tweet,label,raw_tweet,clean_tweet
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline,interesting article--could this explain some of the differences in covid-19 infection rates between countries? does your blood type increase your risk for coronavirus? does your blood type increase your risk for coronavirus? by @healthline
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt,ever wonder what it’s like to be a nurse working on the front lines of covid-19? ---&gt; what it's like to be a nurse working on the front lines of covid-19
2,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)",real,Second Wave COVID19 First Never Ended Many Americans wonder coming months pandemic look like states case rates currently high well states the… Feed generated FetchRSS,"can you have a second wave of covid-19 if the first never ended? many americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the… (feed generated with fetchrss)"
3,Emergency Report! The Coronavirus Hoax Ends Human Civilization https://t.co/iTlBrPcugW #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #...,fake,Emergency Report Coronavirus Hoax Ends Human Civilization SayNoToBillGates NoNewNormal COVID19 Coronavirus Plandemic Scamdemic Fakedemic JailBillGates FuckBillGates BigTechIsEvil 5G Crypto ID2020 Agenda2030 Depopulation WakeUp,emergency report! the coronavirus hoax ends human civilization #saynotobillgates #nonewnormal covid-19 #coronavirus #plandemic #scamdemic #fakedemic #jailbillgates #fuckbillgates #bigtechisevil # g #crypto #id #agenda #depopulation #wakeup
4,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years.",real,Good evidence slow Covid19 mutation rate Bodes well vaccines development people get vaccinated 2021 new coronavirus may well enjoy protection lasts years,"good evidence of a slow covid-19 mutation rate. bodes well for vaccines in development. if people get vaccinated in against the new coronavirus, they may well enjoy a protection that lasts for years."


In [ ]:
tokenizer = CrazyTokenizer(
    keepcaps=True,
    #ignore_stopwords='english',
    #stem = 'stem',
    remove_punct=True, 
    remove_breaks=True,
    decontract=True,
    hashtags='split',
    twitter_handles='realname',
    urls='title',
    normalize=4
)

def concat_tok(text):
  text = tokenizer.tokenize(text)
  text = ' '.join(text)
  text = text.strip()
  return text
data['clean_tweet'] = data.parallel_apply(lambda x: concat_tok(x['clean_tweet']), axis=1)

   0.00%                                          |        0 /     3646 |      
   0.00%                                          |        0 /     3645 |      M   0.99%                                          |       36 /     3646 |      
   0.00%                                          |        0 /     3645 |      M   2.69% :                                        |       98 /     3646 |      
   0.00%                                          |        0 /     3645 |      M   2.74% :                                        |      100 /     3646 |      
   0.00%                                          |        0 /     3645 |      M   2.74% :                              

In [ ]:
data.tail()

,tweet,label,raw_tweet,clean_tweet
7286,"The funded projects include 11 community-led initiatives to address Wisconsin community health needs and ten #UWMadison research grants to lessen the impact of COVID-19 through improved testing, treatments and vaccine development. @WisPartnership\n🔬🧪🩺...",real,funded projects include 11 communityled initiatives address Wisconsin community health needs ten UWMadison research grants lessen impact COVID19 improved testing treatments vaccine development WisPartnership 🔬🧪🩺,the funded projects include community led initiatives to address wisconsin community health needs and ten uw madison research grants to lessen the impact of covid-19 through improved testing treatments and vaccine development test for covid-19 vaccina...
7287,Get it togedffffffffcvhjj\n\nThe CDC Confesses to Lying About COVID-19 Death Numbers https://t.co/vPUi1n3JIw,fake,Get togedffffffffcvhjj CDC Confesses Lying COVID19 Death Numbers,get it togedffffcvhjj the cdc confesses to lying about covid-19 death numbers
7288,"NEW: “If you don’t test, you don’t have any cases,” Trump said at a June 15 roundtable discussion at the White House. “If we stopped testing right now, we’d have very few cases, if any.”\n\n📝: @shefalil, @KHNews/@PolitiFact #HealthCheck https://t.co/Z...",fake,NEW “If don’t test don’t cases” Trump said June 15 roundtable discussion White House “If stopped testing right we’d cases any” 📝 shefalil KHNewsPolitiFact HealthCheck,new if you do not test you do not have any cases trump said at a june roundtable discussion at the white house if we stopped testing right now we would have very few cases if any contact tracing shefali luthra @khnews/ healthcheck trump is take on cov...
7289,COVID-19: The Immune System Can Fight Back https://t.co/UTsavTxqqr via @DubaiChronicle,real,COVID19 Immune System Fight Back via DubaiChronicle,covid-19 the immune system can fight back via
7290,COVID-19: Chinese health authorities confirm patient zero ‘had sex with bats’\n\nTHIS IS DISGUSTING https://t.co/iaYhSCXSjJ,fake,COVID19 Chinese health authorities confirm patient zero ‘had sex bats’ DISGUSTING,covid-19 chinese health authorities confirm patient zero had sex with bats this is disgusting


In [ ]:
def preprocess_again(text):
  text = re.sub(r"[^A-Za-z0-9|-]", " ", text)  
  text = ' '.join(text.split()) 
  return text
data['clean_tweet'] = data['clean_tweet'].apply(lambda s : preprocess_again(s))

In [15]:
data.to_csv(f'{path}additional_data.csv',index=False)

NameError: ignored

In [49]:
additional_data = pd.read_csv(f"{path}additional_data.csv")
additional_data.head()

,tweet,label,raw_tweet,clean_tweet,target
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline,interesting article could this explain some of the differences in covid-19 infection rates between countries does your blood type increase your risk for coronavirus does your blood type increase your risk for coronavirus by,0
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt,ever wonder what it is like to be a nurse working on the front lines of covid-19 what it is like to be a nurse working on the front lines of covid-19,0
2,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)",real,Second Wave COVID19 First Never Ended Many Americans wonder coming months pandemic look like states case rates currently high well states the… Feed generated FetchRSS,can you have a second wave of covid-19 if the first never ended many americans wonder what the coming months of the pandemic will look like in states where case rates are currently high as well as states where the feed generated with fetchrss,0
3,Emergency Report! The Coronavirus Hoax Ends Human Civilization https://t.co/iTlBrPcugW #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #...,fake,Emergency Report Coronavirus Hoax Ends Human Civilization SayNoToBillGates NoNewNormal COVID19 Coronavirus Plandemic Scamdemic Fakedemic JailBillGates FuckBillGates BigTechIsEvil 5G Crypto ID2020 Agenda2030 Depopulation WakeUp,emergency report the coronavirus hoax ends human civilization say no to bill gates no new normal covid-19 coronavirus plan demic scam demic fake demic jail bill gates fuck bill gates big tech is evil g crypto i d agenda depopulation wakeup,1
4,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years.",real,Good evidence slow Covid19 mutation rate Bodes well vaccines development people get vaccinated 2021 new coronavirus may well enjoy protection lasts years,good evidence of a slow covid-19 mutation rate bodes well for vaccines in development if people get vaccinated in against the new coronavirus they may well enjoy a protection that lasts for years,0


In [56]:
# def making_label(st):
#   if(st=='fake'):
#     return 1
#   elif(st=='real'):
#     return 0
# additional_data['target'] = additional_data['label'].apply(making_label)
# additional_data.tail()

In [38]:
additional_data

,tweet,label,raw_tweet,clean_tweet,target
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline,interesting article could this explain some of the differences in covid-19 infection rates between countries does your blood type increase your risk for coronavirus does your blood type increase your risk for coronavirus by,0
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt,ever wonder what it is like to be a nurse working on the front lines of covid-19 what it is like to be a nurse working on the front lines of covid-19,0
2,"Can You Have a Second Wave of COVID-19 If the First Never Ended? Many Americans wonder what the coming months of the pandemic will look like in states where case rates are currently high, as well as states where the…\n\n(Feed generated with FetchRSS)",real,Second Wave COVID19 First Never Ended Many Americans wonder coming months pandemic look like states case rates currently high well states the… Feed generated FetchRSS,can you have a second wave of covid-19 if the first never ended many americans wonder what the coming months of the pandemic will look like in states where case rates are currently high as well as states where the feed generated with fetchrss,0
3,Emergency Report! The Coronavirus Hoax Ends Human Civilization https://t.co/iTlBrPcugW #SayNoToBillGates #NoNewNormal #COVID19 #Coronavirus #Plandemic #Scamdemic #Fakedemic #JailBillGates #FuckBillGates #BigTechIsEvil #5G #Crypto #ID2020 #Agenda2030 #...,fake,Emergency Report Coronavirus Hoax Ends Human Civilization SayNoToBillGates NoNewNormal COVID19 Coronavirus Plandemic Scamdemic Fakedemic JailBillGates FuckBillGates BigTechIsEvil 5G Crypto ID2020 Agenda2030 Depopulation WakeUp,emergency report the coronavirus hoax ends human civilization say no to bill gates no new normal covid-19 coronavirus plan demic scam demic fake demic jail bill gates fuck bill gates big tech is evil g crypto i d agenda depopulation wakeup,1
4,"Good evidence of a slow Covid-19 mutation rate. Bodes well for vaccines in development. If people get vaccinated in 2021 against the new coronavirus, they may well enjoy a protection that lasts for years.",real,Good evidence slow Covid19 mutation rate Bodes well vaccines development people get vaccinated 2021 new coronavirus may well enjoy protection lasts years,good evidence of a slow covid-19 mutation rate bodes well for vaccines in development if people get vaccinated in against the new coronavirus they may well enjoy a protection that lasts for years,0
...,...,...,...,...,...
6556,Wuhan study shows lying face down improves breathing in severe COVID-19 patients and affected people https://t.co/hQPuAcG1n9 via @medical_xpress,real,Wuhan study shows lying face improves breathing severe COVID19 patients affected people via medicalxpress,wuhan study shows lying face down improves breathing in severe covid-19 patients and affected people via,0
6557,"The funded projects include 11 community-led initiatives to address Wisconsin community health needs and ten #UWMadison research grants to lessen the impact of COVID-19 through improved testing, treatments and vaccine development. @WisPartnership\n🔬🧪🩺...",real,funded projects include 11 communityled initiatives address Wisconsin community health needs ten UWMadison research grants lessen impact COVID19 improved testing treatments vaccine development WisPartnership 🔬🧪🩺,the funded projects include community led initiatives to address wisconsin community health needs and ten uw madison research grants to lessen the impact of covid-19 through improved testing trea

In [34]:
# additional_data = additional_data.drop_duplicates(subset=['clean_tweet'],keep='first')
# additional_data = additional_data.reset_index(drop=True)
# additional_data[additional_data['clean_tweet'] == "covid-19 the immune system can fight back via"]

In [51]:
additional_data[additional_data.duplicated(['clean_tweet'])]

,tweet,label,raw_tweet,clean_tweet,target


In [53]:
additional_data.label.value_counts(), len(additional_data)

(real    3957
 fake    2604
 Name: label, dtype: int64, 6561)

In [54]:
additional_data[additional_data.isnull().any(axis=1)]

,tweet,label,raw_tweet,clean_tweet,target


In [60]:
train = pd.read_csv(f"{path}train_data.csv")
train.head(2)

,tweet,label,target,raw_tweet,clean_tweet
0,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.,real,0,The CDC currently reports 99031 deaths. In general the discrepancies in death counts between different sources are small and explicable. The death toll stands at roughly 100000 people today.,the cdc currently reports deaths in general the discrepancies in death counts between different sources are small and explicable the death toll stands at roughly people today
1,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths. https://t.co/YASGRTT4ux,real,0,States reported 1121 deaths a small rise from last Tuesday. Southern states reported 640 of those deaths.,states reported deaths a small rise from last tuesday southern states reported of those deaths


In [59]:
additional_data.head(2)

,tweet,label,raw_tweet,clean_tweet,target
0,Interesting article--could this explain some of the differences in COVID-19 infection rates between countries? Does Your Blood Type Increase Your Risk for Coronavirus? https://t.co/3mhj1urWPf by @healthline,real,Interesting articlecould explain differences COVID19 infection rates countries Blood Type Increase Risk Coronavirus healthline,interesting article could this explain some of the differences in covid-19 infection rates between countries does your blood type increase your risk for coronavirus does your blood type increase your risk for coronavirus by,0
1,Ever wonder what it’s like to be a nurse working on the front lines of COVID-19?\n---&gt; https://t.co/hXYjoAPiLq https://t.co/N46BOvfsgB,real,Ever wonder it’s like nurse working front lines COVID19 gt,ever wonder what it is like to be a nurse working on the front lines of covid-19 what it is like to be a nurse working on the front lines of covid-19,0
